# Bayesian vs Chi-Squared Fitting for ΛCDM using Luminosity Distance

This notebook demonstrates **why Bayesian inference is superior to simple χ² fitting**
when estimating **ΛCDM cosmological parameters** from noisy luminosity distance data.

We will:
- Generate mock supernova-like distance data
- Fit ΛCDM parameters using χ² minimization
- Perform Bayesian inference on the same model
- Compare uncertainties, degeneracies, and interpretations

Level: **Upper undergraduate / early graduate cosmology**

## 1. Cosmological Background

In a flat ΛCDM universe, the Hubble parameter is:

$$H(z) = H_0 \sqrt{\Omega_m (1+z)^3 + (1-\Omega_m)}$$

The luminosity distance is:

$$d_L(z) = (1+z) c \int_0^z \frac{dz'}{H(z')}$$

We will infer:
- Matter density $\Omega_m$
- Hubble constant $H_0$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

c = 3e5  # km/s

def H(z, H0, Om):
    return H0 * np.sqrt(Om * (1+z)**3 + (1-Om))

def luminosity_distance(z, H0, Om):
    z_grid = np.linspace(0, z, 200)
    integrand = c / H(z_grid, H0, Om)
    return (1+z) * np.trapz(integrand, z_grid)

## 2. Generate Mock Supernova Data

We generate noisy luminosity distance measurements similar to Type Ia supernova data.

In [ ]:
np.random.seed(0)

H0_true = 70
Om_true = 0.3

z = np.linspace(0.01, 1.0, 25)
dL_true = np.array([luminosity_distance(zi, H0_true, Om_true) for zi in z])

sigma = 200  # Mpc uncertainty
dL_obs = dL_true + np.random.normal(0, sigma, size=len(z))

plt.errorbar(z, dL_obs, yerr=sigma, fmt='o', label='Data')
plt.plot(z, dL_true, label='True model')
plt.xlabel('Redshift z')
plt.ylabel('Luminosity Distance (Mpc)')
plt.legend()
plt.show()

## 3. χ² Fitting

We first perform a traditional χ² minimization to find the best-fit parameters.

In [ ]:
def chi2(H0, Om):
    model = np.array([luminosity_distance(zi, H0, Om) for zi in z])
    return np.sum((dL_obs - model)**2 / sigma**2)

H0_grid = np.linspace(60, 80, 60)
Om_grid = np.linspace(0.1, 0.5, 60)

chi2_map = np.zeros((len(H0_grid), len(Om_grid)))

for i, H0v in enumerate(H0_grid):
    for j, Omv in enumerate(Om_grid):
        chi2_map[i, j] = chi2(H0v, Omv)

i_min, j_min = np.unravel_index(np.argmin(chi2_map), chi2_map.shape)
H0_best, Om_best = H0_grid[i_min], Om_grid[j_min]

print(f'Chi-squared best fit: H0 = {H0_best:.1f}, Omega_m = {Om_best:.2f}')

### Limitation of χ²

χ² gives **one best-fit point**, but tells us little about:
- Parameter degeneracies
- Non-Gaussian uncertainties
- Physical plausibility beyond the minimum

## 4. Bayesian Inference

We now compute the **posterior distribution**:

$$P(H_0, \Omega_m | D) \propto \exp(-\chi^2/2) P(H_0) P(\Omega_m)$$

We use uniform, physically motivated priors.

In [ ]:
posterior = np.exp(-0.5 * (chi2_map - chi2_map.min()))
posterior /= np.sum(posterior)

# Marginalize
P_H0 = np.sum(posterior, axis=1)
P_Om = np.sum(posterior, axis=0)

H0_mean = np.sum(H0_grid * P_H0)
Om_mean = np.sum(Om_grid * P_Om)

print(f'Bayesian mean estimates: H0 = {H0_mean:.1f}, Omega_m = {Om_mean:.2f}')

In [ ]:
plt.contourf(Om_grid, H0_grid, posterior, levels=30)
plt.xlabel('$\Omega_m$')
plt.ylabel('$H_0$')
plt.title('Bayesian Posterior Distribution')
plt.colorbar(label='Posterior probability')
plt.show()

## 5. Why Bayesian Inference Wins

From the posterior we learn:
- $H_0$ and $\Omega_m$ are **correlated**
- There is a **degeneracy direction** invisible to χ²
- Uncertainties are **non-elliptical**

Bayesian inference preserves the *full information content* of the data.

## 6. Key Takeaway

**χ² fitting finds a point. Bayesian inference finds understanding.**

For ΛCDM and modern cosmology, Bayesian methods are essential.